# Install Dependencies

In [37]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


# Import Dependencies

In [38]:
import mediapipe as mp
import cv2

# Initialize MediaPipe

In [39]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Get Live Webcam Feed

In [60]:
results.face_landmarks.landmark[1]

x: 0.5227743
y: 0.5018029
z: -0.045808684

In [62]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks[0,1], mp_holistic.FACEMESH_TESSELATION)
        
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

TypeError: 'NormalizedLandmarkList' object is not subscriptable

: 